#### Apply TinyML techniques to reduce model sizes

Optimize the model using TensorFlow Lite and quantization. Convert the trained model to the TensorFlow Lite format and then apply quantization to reduce its size.

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# Load back X_train, X_test, y_train, and y_test
X_train = np.loadtxt('train_data/X_train_t10.txt')
X_test = np.loadtxt('train_data/X_test_t10.txt')
y_train = np.loadtxt('train_data/y_train_t10.txt')
y_test = np.loadtxt('train_data/y_test_t10.txt')

In [3]:
X_train_scaled = X_train
X_test_scaled = X_test

Create a Deep Neural-Networks

In [4]:
# Model Selection and Training
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

#### Model Training

In [5]:
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
5/5 [==============================] - 1s 43ms/step - loss: 4752.9922 - accuracy: 0.5538 - val_loss: 1274.3019 - val_accuracy: 0.5455
Epoch 2/10
5/5 [==============================] - 0s 13ms/step - loss: 1008.7768 - accuracy: 0.4769 - val_loss: 1079.9604 - val_accuracy: 0.4545
Epoch 3/10
5/5 [==============================] - 0s 13ms/step - loss: 850.8342 - accuracy: 0.5231 - val_loss: 322.9302 - val_accuracy: 0.4545
Epoch 4/10
5/5 [==============================] - 0s 15ms/step - loss: 830.6862 - accuracy: 0.4154 - val_loss: 1181.3683 - val_accuracy: 0.5455
Epoch 5/10
5/5 [==============================] - 0s 13ms/step - loss: 760.0226 - accuracy: 0.4923 - val_loss: 437.0700 - val_accuracy: 0.4545
Epoch 6/10
5/5 [==============================] - 0s 13ms/step - loss: 338.5021 - accuracy: 0.5692 - val_loss: 807.1042 - val_accuracy: 0.5455
Epoch 7/10
5/5 [==============================] - 0s 14ms/step - loss: 574.1248 - accuracy: 0.4308 - val_loss: 261.2747 - val_accuracy: 0

Model Evaluation

In [6]:
# Model Evaluation
predictions = model.predict(X_test_scaled)
predictions_binary = (predictions > 0.5).astype(int)

2/2 [==============================] - 0s 3ms/step


In [7]:
print(classification_report(y_test, predictions_binary))

              precision    recall  f1-score   support

         0.0       0.54      1.00      0.70        22
         1.0       0.00      0.00      0.00        19

    accuracy                           0.54        41
   macro avg       0.27      0.50      0.35        41
weighted avg       0.29      0.54      0.37        41



In [8]:
# Convert and save the TensorFlow model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\theor\AppData\Local\Temp\tmpdm63kev4\assets


Save Model

In [9]:
# Save the TensorFlow Lite model to a file
model_file_path = os.path.join('models', 'dnn_model.tflite')
with open(model_file_path, "wb") as f:
    f.write(tflite_model)

In [10]:
model_size = os.path.getsize(model_file_path)
print("Model is %d bytes" % model_size)

Model is 39358480 bytes


Optimize the model

In [11]:
# Set optimizations (quantization)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Convert the model
tflite_model_optimize = converter.convert()

model_file_path = os.path.join('models', 'model_optimize.tflite')
with open(model_file_path, "wb") as f:
    f.write(tflite_model_optimize)

INFO:tensorflow:Assets written to: C:\Users\theor\AppData\Local\Temp\tmp66h_gopk\assets


INFO:tensorflow:Assets written to: C:\Users\theor\AppData\Local\Temp\tmp66h_gopk\assets


In [12]:
optimized_model_size = os.path.getsize(model_file_path)
print("Optimized Model is %d bytes" % optimized_model_size)

Optimized Model is 9842064 bytes


Evaluate the optimized model

In [13]:
# Load the optimized TensorFlow Lite model
optimized_model_path = 'models/model_optimize.tflite'
interpreter = tf.lite.Interpreter(model_path=optimized_model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input data
input_data = X_test_scaled[0:1].astype(np.float32)  # Take the first sample from X_test_scaled

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Convert output to binary predictions
predictions_binary = (output_data > 0.5).astype(int)

# Evaluate the predictions
print(classification_report(y_test[0:1], predictions_binary))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



#### Encode the Model to a Header File 

The next cell creates a constant byte array that contains the TFlite model. Import it as a tab with the sketch below.

In [14]:
# Define the input and output file paths
input_file = 'tflite_model_optimize'
output_file = './model_deployment/include/model.h'

# Write the beginning of the model.h file
with open(output_file, 'w') as f:
    f.write("const unsigned char model[] = {\n")

# Open file in append binary mode
with open(output_file, 'ab') as f:
    # Convert binary data to hexadecimal representation and write to the output file
    hex_data = ', '.join("0x{:02x}".format(byte) for byte in tflite_model_optimize)
    f.write(bytes(hex_data, 'utf-8'))

# Append the end of the model.h file
with open(output_file, 'a') as f:
    f.write("\n};\n")